# core

In [ ]:
#| default_exp core

In [ ]:
#| export
from fastcore.all import *
from yt_dlp import YoutubeDL
import json

In [ ]:
#| hide
from nbdev.showdoc import *

## Videos

In [ ]:
video_url = 'https://www.youtube.com/watch?v=8SF_h3xF3cE'

In [ ]:
#| export
class YTVideo:
    def __init__(self, data:dict): store_attr()
    def __repr__(self):
        flds = ['title']
        sig = ', '.join(f'{o}={self.data[o]!r}' for o in flds if o in self.data)
        return f'YTVideo({sig})'

YTVideo(res)

YTVideo(title='Practical Deep Learning for Coders: Lesson 1')

In [ ]:
#| export
@patch(cls_method=True)
def from_url(cls:YTPlaylist, url:str, quiet:bool=True)->YTPlaylist:
    with YoutubeDL({'flat_playlist':True, 'extract_flat':True, 'quiet':quiet}) as ydl:
        data = ydl.extract_info(url, download=False)
        return YTPlaylist(data)

In [ ]:
opts = {
    'writedescription': True,
    'writesubtitles': True,
    'writeautomaticsub': True,
    'quiet': True,
    'no_warnings': True,
    'skip_download': True,
    'subtitlesformat': 'srt'
}

with YoutubeDL(opts) as ydl:
    res = ydl.extract_info(video_url, download=False)

In [ ]:
res.keys()

dict_keys(['id', 'title', 'formats', 'thumbnails', 'thumbnail', 'description', 'channel_id', 'channel_url', 'duration', 'view_count', 'average_rating', 'age_limit', 'webpage_url', 'categories', 'tags', 'playable_in_embed', 'live_status', 'media_type', 'release_timestamp', '_format_sort_fields', 'automatic_captions', 'subtitles', 'comment_count', 'chapters', 'heatmap', 'like_count', 'channel', 'channel_follower_count', 'creators', 'license', 'uploader', 'uploader_id', 'uploader_url', 'upload_date', 'timestamp', 'availability', 'original_url', 'webpage_url_basename', 'webpage_url_domain', 'extractor', 'extractor_key', 'playlist', 'playlist_index', 'display_id', 'fulltitle', 'duration_string', 'release_year', 'is_live', 'was_live', 'requested_subtitles', '_has_drm', 'epoch', 'requested_formats', 'format', 'format_id', 'ext', 'protocol', 'language', 'format_note', 'filesize_approx', 'tbr', 'width', 'height', 'resolution', 'fps', 'dynamic_range', 'vcodec', 'vbr', 'stretched_ratio', 'aspect_

In [ ]:
{k:v for k,v in res.items() if k not in ['formats','thumbnails','automatic_captions', 'subtitles', 'chapters', 'heatmap', 'requested_formats']}

{'id': '8SF_h3xF3cE',
 'title': 'Practical Deep Learning for Coders: Lesson 1',
 'thumbnail': 'https://i.ytimg.com/vi_webp/8SF_h3xF3cE/maxresdefault.webp',
 'description': 'Go to https://course.fast.ai for code, notebooks, quizzes, etc. This course is designed for people with some coding experience who want to learn how to apply deep learning and machine learning to practical problems. There are 9 lessons, and each lesson is around 90 minutes long.\n\nWe cover topics such as how to:\n- Build and train deep learning, random forest, and regression models\n- Deploy models\n- Apply deep learning to computer vision, natural language processing, tabular analysis, and collaborative filtering problems\n- Use PyTorch, the world’s fastest growing deep learning software, together with popular libraries such as fastai, Hugging Face Transformers, and gradio\n\nYou don’t need any special hardware or software — we’ll show you how to use free resources for both building and deploying models. You don’t

In [ ]:
res['webpage_url']

'https://www.youtube.com/watch?v=8SF_h3xF3cE'

## Playlists

In [ ]:
playlist_url = 'https://www.youtube.com/playlist?list=PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU'

In [ ]:
#| export
class YTPlaylist:
    def __init__(self, data:dict): store_attr()
    def __repr__(self):
        flds = ['webpage_url', 'title', 'channel', 'playlist_count']
        sig = ', '.join(f'{o}={self.data[o]!r}' for o in flds)
        return f'YTPlaylist({sig})'

In [ ]:
#| export
@patch(cls_method=True)
def from_url(cls:YTPlaylist, url:str, quiet:bool=True)->YTPlaylist:
    with YoutubeDL({'flat_playlist':True, 'extract_flat':True, 'quiet':quiet}) as ydl:
        data = ydl.extract_info(url, download=False)
        return YTPlaylist(data)

In [ ]:
playlist = YTPlaylist.from_url(playlist_url)
playlist

YTPlaylist(webpage_url='https://www.youtube.com/playlist?list=PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU', title='Practical Deep Learning for Coders', channel='Jeremy Howard', playlist_count=8)

In [ ]:
playlist.data['description']

In [ ]:
playlist.data['entries'][0]

{'_type': 'url',
 'ie_key': 'Youtube',
 'id': '8SF_h3xF3cE',
 'url': 'https://www.youtube.com/watch?v=8SF_h3xF3cE',
 'title': 'Practical Deep Learning for Coders: Lesson 1',
 'description': None,
 'duration': 4976,
 'channel_id': None,
 'channel': None,
 'channel_url': None,
 'uploader': None,
 'uploader_id': None,
 'uploader_url': None,
 'thumbnails': [{'url': 'https://i.ytimg.com/vi/8SF_h3xF3cE/hqdefault.jpg?sqp=-oaymwEbCKgBEF5IVfKriqkDDggBFQAAiEIYAXABwAEG&rs=AOn4CLDzGS4dCrE4P5ZXNZfLHs7PWgtB7g',
   'height': 94,
   'width': 168},
  {'url': 'https://i.ytimg.com/vi/8SF_h3xF3cE/hqdefault.jpg?sqp=-oaymwEbCMQBEG5IVfKriqkDDggBFQAAiEIYAXABwAEG&rs=AOn4CLCzscwuCifqJOpZaat71ITGGBbp6A',
   'height': 110,
   'width': 196},
  {'url': 'https://i.ytimg.com/vi/8SF_h3xF3cE/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLBGEE8rwgRWK3J7GRU1tFnqjWysIg',
   'height': 138,
   'width': 246},
  {'url': 'https://i.ytimg.com/vi/8SF_h3xF3cE/hqdefault.jpg?sqp=-oaymwEcCNACELwBSFXyq4qp

# -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()